In [1]:
import gradio as gr
import pandas as pd
import mlflow
import numpy as np
import mlflow.pyfunc
import xgboost as xgb
import mlflow.xgboost
import pickle
from gradio_functions import *
import matplotlib.pyplot as plt
import plotly.express as px
from model_functions import *
import html
mlflow.set_tracking_uri("http://localhost:5000")

c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\michi\AppData\Roaming\Python\Python38\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [2]:
with gr.Blocks() as demo:
    with gr.Tab(label="Immopreisrechner"):
        gr.Markdown(
            """
        # Geben Sie hier an, welche Merkmale Ihre Immobilie besitzt 🏠
        """
        )
        with gr.Row():
            feature_zip = gr.Dropdown(
                label="Postleitzahl",
                choices=[
                    "97070 Würzburg-Altstadt",
                    "97072 Würzburg-Sanderau",
                    "97074 Würzburg-Frauenland",
                    "97076 Würzburg-Lengfeld",
                    "97078 Würzburg-Lindleinsmühle",
                    "97080 Würzburg-Dürrbachtal",
                    "97082 Würzburg-Steinbachtal",
                    "97084 Würzburg-Heuchelhof",
                    "97204 Höchberg",
                    "97209 Veitshöchheim",
                    "97218 Gerbrunn",
                    "97222 Rimpar",
                    "97228 Rottendorf",
                    "97234 Reichenberg",
                    "97236 Randersacker",
                    "97246 Eibelstadt",
                    "97249 Eisingen",
                    "97250 Erlabrunn",
                    "97261 Güntersleben",
                    "97270 Kist",
                    "97288 Theilheim",
                    "97297 Waldbüttelbrunn",
                    "97299 Zell am Main",
                ],
                value="97070 Würzburg-Altstadt",
            )
        with gr.Row():
            feature_squrmeter = gr.Number(label="Wohnfläche [qm]", value=79)
        with gr.Row():
            feature_rooms = gr.Number(label="Anzahl der Zimmer", value=3)
        with gr.Row():
            features_altbau = gr.Checkbox(label="Altbau (bis 1945)")
            feature_balkon = gr.Checkbox(label="Balkon")
            feature_ba = gr.Checkbox(label="Barrierefrei")
            feature_dachgeschoss = gr.Checkbox(label="Dachgeschoss")
            feature_einbaukueche = gr.Checkbox(label="Einbauküche")
            feature_neubau = gr.Checkbox(label="Neubau")
            feature_parkett = gr.Checkbox(label="Parkett")
            feature_stellplatz = gr.Checkbox(label="Stellplatz")
            feature_badwc_getrennt = gr.Checkbox(label="Bad/WC getrennt")
            feature_personenaufzug = gr.Checkbox(label="Personenaufzug")
            feature_garten = gr.Checkbox(label="Garten")
            feature_garage = gr.Checkbox(label="Garage")
            feature_renoviert = gr.Checkbox(label="Renoviert")
            feature_terrasse = gr.Checkbox(label="Terrasse")
            feature_wanne = gr.Checkbox(label="Badewanne")
            feature_zentralheizung = gr.Checkbox(label="Zentralheizung")
            feature_abstellraum = gr.Checkbox(label="Abstellraum")
            feature_fernwaerme = gr.Checkbox(label="Fernwärme")
            feature_fussbodenheizung = gr.Checkbox(label="Fußbodenheizung")
            feature_gartenmitbenutzung = gr.Checkbox(label="Gartenmitbenutzung")
            feature_kellerabteil = gr.Checkbox(label="Kellerabteil")

        with gr.Accordion("erweiterte Optionen", open=False):
            erklärung = gr.Checkbox(label="XGBoost vorhersagen")

        btn = gr.Button("Berechne deinen Preis")
        output_html = gr.HTML(label="Dein Preis")
        btn.click(
            fn=trigger_actions,
            inputs=[
                feature_squrmeter,
                feature_zip,
                feature_rooms,
                features_altbau,
                feature_balkon,
                feature_ba,
                feature_dachgeschoss,
                feature_einbaukueche,
                feature_neubau,
                feature_parkett,
                feature_stellplatz,
                feature_badwc_getrennt,
                feature_personenaufzug,
                feature_garten,
                feature_garage,
                feature_renoviert,
                feature_terrasse,
                feature_wanne,
                feature_zentralheizung,
                feature_abstellraum,
                feature_fernwaerme,
                feature_fussbodenheizung,
                feature_gartenmitbenutzung,
                feature_kellerabteil,
                erklärung,
            ],
            outputs=[output_html],
        )

    with gr.Tab(label="Admin"):
        gr.Markdown(
            """
        # Admin-Seite
        Hier können Sie Ihre Modelle nachtrainieren und die verschiedenen Modelle vergleichen.
        """
        )
        gr.Markdown("## Datengrundlage erkunden und verstehen:")
        link = "https://michaelseitz98.github.io/enterprise-ai-project/eda-wue-rent-all.html"
        description = (
            "Datenanalyse des Ausgangsdatensatz für Würzburger Wohnungen (Miete), EDA"
        )
        html_code = f'<a href="{html.escape(link)}" font-size: 16px;">{html.escape(description)}</a>'
        gr.HTML(html_code)
        link = (
            "https://michaelseitz98.github.io/enterprise-ai-project/eda-wue-houses.html"
        )
        description = "Jetzt anschauen: Datenanalyse des Ausgangsdatensatz Würzburger Häuser (Kauf), EDA"
        html_code = f'<a href="{html.escape(link)}" font-size: 16px;">{html.escape(description)}</a>'
        gr.HTML(html_code)
        gr.HTML("<hr>")

        gr.Markdown("## Dynamisches Nachtraining")
        gr.Markdown("Wählen Sie die Modelle aus, die Sie nachtrainieren möchten und miteinander vergleichen möchten")

        with gr.Row():
            xgb = gr.Checkbox(label="xgb")
            rf = gr.Checkbox(label="rf")
            linear = gr.Checkbox(label="linear", value=True)
            lasso = gr.Checkbox(label="lasso")
            ridge = gr.Checkbox(label="ridge")
            elasticnet = gr.Checkbox(label="elasticnet")
            baseline = gr.Checkbox(label="baseline-rent", value=True)

        model_list = []
        if xgb:
            model_list.append("xgb")
        if rf:
            model_list.append("rf")
        if lasso:
            model_list.append("lasso")
        if ridge:
            model_list.append("ridge")
        if elasticnet:
            model_list.append("elasticnet")
        if baseline:
            model_list.append("baseline-rent")

        limit = gr.Number(label="Anzahl der zu scrapenden Seiten ", value=1, maximum=10)
        nachtraining = gr.Button("Aktuelle Daten Scrapen, Modelle nachtrainieren und evaluieren")
        
        df_results = gr.Dataframe(label="Ergebnisse Nachtraining")
        output_plot = gr.Plot(visible=False)

        nachtraining.click(fn=gradio_retrain_with_added_data, inputs=[xgb, ridge, rf, elasticnet, linear, lasso, baseline, limit], outputs=[df_results, output_plot])

demo.queue(concurrency_count=10).launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [ ]:
import gradio as gr
import time

def slowly_reverse(word, progress=gr.Progress()):
    progress(0, desc="Starting")
    time.sleep(1)
    progress(0.5, desc="Halfway")
    time.sleep(1)
    new_string = ""
    for letter in progress.tqdm(word, desc="Reversing"):
        time.sleep(0.25)
        new_string = letter + new_string
    return new_string

demo = gr.Interface(slowly_reverse, gr.Text(), gr.Text())

if __name__ == "__main__":
    demo.queue(concurrency_count=10).launch()